## TraceMethod
创建路径，思路是依据 [在 Tableau 中创建显示起点和终点之间的路径的地图](https://onlinehelp.tableau.com/current/pro/desktop/zh-cn/maps_howto_origin_destination.htm) 指导来做的。依据该该思路来完成创建路径图演示，需要的数据：
* 路径序列号——表示是第几个路径，用于区分路线
* 点顺序——用于表示每条路径上的点顺序，用于区分每条路线上的先后顺序的点
* 经纬度——经纬度数据，尽量在此处处理完全

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time
%matplotlib inline

In [2]:
df = pd.read_csv("./MobikeData/mobike_shanghai_sample_updated.csv")

In [3]:
df.head(2)

,orderid,bikeid,userid,start_time,start_location_x,start_location_y,end_time,end_location_x,end_location_y,track
0,78387,158357,10080,2016-08-20 06:57,121.348,31.389,2016-08-20 07:04,121.357,31.388,"121.347,31.392#121.348,31.389#121.349,31.390#1..."
1,891333,92776,6605,2016-08-29 19:09,121.508,31.279,2016-08-29 19:31,121.489,31.271,"121.489,31.270#121.489,31.271#121.490,31.270#1..."


In [4]:
# 复制需要用于演示的 5 行数据
data = df.head(5).copy()

In [7]:
# 拆分 track 获取每个路径上的点
data = pd.concat([data, data.track.str.split("#", expand=True)], axis=1)

In [10]:
# 将每个 track 上的路径点和路径合并，思路上需要将 index 设置为路径编号
data = data.reset_index()
data.rename({"index": "trace"}, axis=1,inplace=True)

In [11]:
data.head(2)

,level_0,trace,orderid,bikeid,userid,start_time,start_location_x,start_location_y,end_time,end_location_x,...,31,32,33,34,35,36,37,38,39,40
0,0,0,78387,158357,10080,2016-08-20 06:57,121.348,31.389,2016-08-20 07:04,121.357,...,None,None,None,None,None,None,None,None,None,None
1,1,1,891333,92776,6605,2016-08-29 19:09,121.508,31.279,2016-08-29 19:31,121.489,...,"121.506,31.277","121.507,31.277","121.507,31.278","121.507,31.279","121.508,31.279",None,None,None,None,None


In [13]:
# 转换数据，长数据转宽数据；同时删除缺失值，减少不必要数据
data = data.melt(id_vars=["trace"], value_vars=np.r_[0:40], var_name="traceOrder", value_name="location").dropna()

In [16]:
# 获取经纬度数据
data[["longitude", "latitude"]] = data.location.str.split(",", expand=True)

In [17]:
# 检查数据以及保存数据
data.head()

,trace,traceOrder,location,longitude,latitude
0,0,0,"121.347,31.392",121.347,31.392
1,1,0,"121.489,31.270",121.489,31.270
2,2,0,"121.381,31.251",121.381,31.251
3,3,0,"121.471,31.325",121.471,31.325
4,4,0,"121.407,31.291",121.407,31.291


In [18]:
data.to_csv("./data/mobike_trace.csv", index=False)

## 接下来使用 Tableau 进行绘制路径图

![](https://ws4.sinaimg.cn/large/006tKfTcly1g0tb4g5kz8j31gi0u01cg.jpg)